In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import text  # Importar text
import os
from dotenv import load_dotenv
import time

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las credenciales de la base de datos desde las variables de entorno
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = int(os.getenv('DB_PORT', 3306))  # Asegúrate de convertir a entero
db_name = os.getenv('DB_NAME')

# Función para conectar a la base de datos
def connect_to_db():
    try:
        engine = sqlalchemy.create_engine(
            f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}',
            connect_args={'connect_timeout': 30}  # Ajustar el tiempo de espera de conexión
        )
        connection = engine.connect()
        print("Conexión a la base de datos establecida con éxito.")
        return engine, connection
    except Exception as e:
        print(f"Error al conectar a la base de datos: {e}")
        raise

# Conectar a MySQL utilizando las credenciales del archivo .env
engine, connection = connect_to_db()

# Consulta para eliminar los registros deseados
try:
    # Definir las fechas de inicio y fin del mes deseado
    start_date = '2024-07-01 00:00:00'
    end_date = '2024-07-31 23:59:59'

    # Verificar cuántos registros serán eliminados antes de ejecutar el DELETE
    select_query = text("""
        SELECT COUNT(*) FROM taxi_fhv_data
        WHERE source LIKE :source_pattern
        AND Pickup_datetime BETWEEN :start_date AND :end_date;
    """)
    result = connection.execute(select_query, {"source_pattern": "yellow%", "start_date": start_date, "end_date": end_date})
    count = result.fetchone()[0]
    print(f"Registros que se eliminarán: {count}")

    # Confirmación antes de eliminar
    confirm = input("¿Deseas continuar con la eliminación? (sí/no): ").strip().lower()
    if confirm == 'si':
        delete_query = text("""
            DELETE FROM taxi_fhv_data
            WHERE source LIKE :source_pattern
            AND Pickup_datetime BETWEEN :start_date AND :end_date;
        """)
        result = connection.execute(delete_query, {"source_pattern": "yellow%", "start_date": start_date, "end_date": end_date})
        print(f"Registros eliminados: {result.rowcount}")
    else:
        print("Operación cancelada por el usuario.")

except Exception as e:
    print(f"Error durante la ejecución de la consulta: {e}")
finally:
    connection.close()
    engine.dispose()


Conexión a la base de datos establecida con éxito.
Registros que se eliminarán: 1499986
Registros eliminados: 1499986


🗃️ Database Cleanup Script: MySQL Table Reset Tool 🧹

In [ ]:
import pandas as pd
import sqlalchemy
from sqlalchemy import text
import os
from dotenv import load_dotenv

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las credenciales de la base de datos desde las variables de entorno
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = int(os.getenv('DB_PORT', 3306))  # Asegúrate de convertir a entero
db_name = os.getenv('DB_NAME')

# Función para conectar a la base de datos
def connect_to_db():
    try:
        engine = sqlalchemy.create_engine(
            f'mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}',
            connect_args={'connect_timeout': 30}  # Ajustar el tiempo de espera de conexión
        )
        connection = engine.connect()
        print("Conexión a la base de datos establecida con éxito.")
        return engine, connection
    except Exception as e:
        print(f"Error al conectar a la base de datos: {e}")
        raise

# Conectar a MySQL utilizando las credenciales del archivo .env
engine, connection = connect_to_db()

# Borrar todas las filas de la tabla 'taxis_vfh_data'
try:
    delete_query = text("DELETE FROM taxis_fhv_data")
    connection.execute(delete_query)
    print("Todas las filas de la tabla 'taxis_vfh_data' han sido eliminadas.")
except Exception as e:
    print(f"Error al borrar las filas de la tabla: {e}")
finally:
    connection.close()
    engine.dispose()


Conexión a la base de datos establecida con éxito.
Error al borrar las filas de la tabla: (pymysql.err.ProgrammingError) (1146, "Table 'UrbanTransit.taxis_fhv_data' doesn't exist")
[SQL: DELETE FROM taxis_fhv_data]
(Background on this error at: https://sqlalche.me/e/20/f405)


🚖 NYC Taxi Zones Data Uploader: CSV to MySQL Migration Tool 📊

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import mysql.connector
from datetime import datetime

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las variables de entorno
db_host = os.getenv('DB_HOST')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')
db_port = os.getenv('DB_PORT')
csv_file_path = 'Automated_Traffic_Volume_Counts.csv'  # Asegúrate de actualizar esto con tu archivo CSV correcto

# Verificar si las variables de entorno se cargaron correctamente
if not all([db_host, db_user, db_password, db_name, db_port]):
    raise ValueError("No se pudieron cargar todas las variables de entorno. Por favor verifica el archivo .env.")

# Cargar el archivo CSV en un DataFrame de pandas
try:
    df = pd.read_csv(csv_file_path)
except FileNotFoundError as e:
    raise FileNotFoundError(f"El archivo {csv_file_path} no se encuentra. Por favor verifica la ruta.")

# Filtrar los datos desde enero 2023 hasta agosto 2024
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 8, 31)
df['datetime'] = pd.to_datetime(df[['Yr', 'M', 'D', 'HH', 'MM']])
df_filtered = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)].copy()

# Seleccionar solo las columnas necesarias para el análisis
columns_to_keep = ['RequestID', 'Boro', 'Yr', 'M', 'D', 'HH', 'MM', 'Vol', 'SegmentID', 'WktGeom', 'street', 'Direction']
df_filtered = df_filtered[columns_to_keep]

# Conectar a la base de datos MySQL
try:
    db_connection = mysql.connector.connect(
        host=db_host,
        user=db_user,
        password=db_password,
        database=db_name,
        port=int(db_port)
    )
except mysql.connector.Error as err:
    raise mysql.connector.Error(f"Error al conectar a la base de datos: {err}")

cursor = db_connection.cursor()

# Insertar los datos filtrados del DataFrame en bloques de 100,000 filas
chunk_size = 100000
rows_inserted = 0
for start in range(0, len(df_filtered), chunk_size):
    end = start + chunk_size
    chunk = df_filtered.iloc[start:end]

    insert_query = """
    INSERT INTO trafico (RequestID, Boro, Yr, M, D, HH, MM, Vol, SegmentID, WktGeom, street, Direction)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    data = [(
        row['RequestID'], row['Boro'], row['Yr'], row['M'], row['D'], row['HH'], row['MM'], row['Vol'],
        row['SegmentID'], row['WktGeom'], row['street'], row['Direction']
    ) for _, row in chunk.iterrows()]

    cursor.executemany(insert_query, data)
    rows_inserted += len(data)
    
    # Imprimir el progreso solo cada 10 bloques
    if start // chunk_size % 10 == 0:
        print(f"Se han insertado {len(data)} filas en este bloque. Total filas insertadas: {rows_inserted}")

# Confirmar los cambios y cerrar la conexión
db_connection.commit()
cursor.close()
db_connection.close()

# Mostrar el número total de filas insertadas
print(f"Se han insertado un total de {rows_inserted} filas en la base de datos.")


   RequestID    Boro    Yr  M   D  HH  MM  Vol  SegmentID  \
0      32970  Queens  2021  4  30   2   0    0     149701   
1      32970  Queens  2021  4  30   2  15    1     149701   
2      32970  Queens  2021  4  30   2  30    0     149701   
3      32970  Queens  2021  4  30   2  45    0     149701   
4      32970  Queens  2021  4  30   3   0    1     149701   

                                        WktGeom          street  \
0  POINT (997407.0998491726 208620.92612708386)  PULASKI BRIDGE   
1  POINT (997407.0998491726 208620.92612708386)  PULASKI BRIDGE   
2  POINT (997407.0998491726 208620.92612708386)  PULASKI BRIDGE   
3  POINT (997407.0998491726 208620.92612708386)  PULASKI BRIDGE   
4  POINT (997407.0998491726 208620.92612708386)  PULASKI BRIDGE   

                    fromSt      toSt Direction  
0  Newtown Creek Shoreline  Dead end        NB  
1  Newtown Creek Shoreline  Dead end        NB  
2  Newtown Creek Shoreline  Dead end        NB  
3  Newtown Creek Shoreline  Dead e